импорты и константы

In [1]:
import os
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
import re
import shutil
import time

# 153.8336486816406250 884.2712384333286764    измеряемый диапазон. 0-2136 диапазон данных
start = 400  # нм
end = 700  # нм
step = (884.2712384333286764 - 153.8336486816406250) / 2136

# номера точек в массиве по координате х
start_point = round((start - 153.8336486816406250) / step)
end_point = start_point + int((end - start) / step)
_start = start_point + 11
_end = start_point + round((end_point - start_point) / 2)

функции получения данных

In [2]:
def get_max_rmr(path, fl):
    _dm = np.zeros(len(fl))

    for i in range(len(fl)):
        with open(path + fl[i], "r", encoding="utf8") as spec:
            _dm[i] = max(re.split(",", spec.read())[_start:_end])
    return _dm

получение максимумов из спектров

In [9]:
data_max = np.zeros(1)  # изанчальные графики
main_folder = r"C:\Users\Nik\Desktop\скрининг\4-25С"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
print(main_folder)
file_list = np.array(natsorted(os.listdir(main_folder)))
for i in range(0, len(file_list), 5000):
    _b = i + 5000
    if _b > len(file_list):
        _b = len(file_list)
    data_max = np.append(data_max, get_max_rmr(main_folder, file_list[i:_b]), axis=0)
    print("получено ", _b)
print("получено " + str(len(data_max)) + " максимумов спектров")

C:/Users/Nik/Desktop/скрининг/4-25С/
получено  5000
получено  10000
получено  15000
получено  20000
получено  25000
получено  30000
получено  35000
получено  40000
получено  45000
получено  50000
получено  55000
получено  60000
получено  65000
получено  70000
получено  75000
получено  80000
получено  85000
получено  90000
получено  95000
получено  100000
получено  104825
получено 104826 максимумов спектров


максимумы на таймлайнах

In [10]:
_step = 400
_sp = 0
_ep = _step

# создание папки для сохранения картинок
dist_folder = r"C:\Users\Nik\Desktop\скрининг\timelines"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"
folder_name = "4"
if not os.path.exists(os.path.join(dist_folder, folder_name)):
    os.makedirs(os.path.join(dist_folder, folder_name))

# расчет гистограммы
num_bins = 10  # количество интервалов
range_values = (np.max(data_max) - np.min(data_max)) / num_bins
# границы интервалов
bin_edges = np.arange(np.min(data_max), np.max(data_max) + range_values, range_values)
# середины интервалов
bins = bin_edges[:-1] + range_values / 2

hist, _ = np.histogram(data_max, bins=bin_edges)

zero = bin_edges[np.argmax(hist) + 1]
fig, ax = plt.subplots(figsize=(10, 6), dpi=120)
fig.suptitle("гистограмма")
ax.hist(data_max, bin_edges, alpha=0.6, color="blue", edgecolor="k", linewidth=1)
ax.plot(bins, hist, color="red", lw=2)
plt.xticks(rotation=90)
ax.set_xticks(bin_edges)
plt.savefig(dist_folder + "/" + folder_name + "/" + "гистограмма.png")

m = np.min(data_max) * 0.8
m2 = np.max(data_max) * 1.1

for i in range(round(len(data_max) / (3 * _step))):
    fig, axs = plt.subplots(3, 1, figsize=(10, 6), dpi=120)
    plt.ioff()
    _s = _sp
    for j in range(3):
        if _ep > len(data_max):
            _ep = len(data_max)
        axs[j].plot(np.arange(_sp, _ep, 1), data_max[_sp:_ep], lw=1, color="blue")
        axs[j].scatter(np.arange(_sp, _ep, 1), data_max[_sp:_ep], s=4, color="black")
        axs[j].plot((_sp, _ep), (zero, zero), color="orange")
        axs[j].set_ylim(m, m2)
        _sp += _step
        _ep += _step

    _e = _ep - _step
    fig.suptitle("спектры " + str(_s) + "-" + str(_e))
    plt.savefig(
        dist_folder
        + "/"
        + folder_name
        + "/"
        + "ряд максимумов "
        + str(_s)
        + "-"
        + str(_e)
        + ".png"
    )

In [12]:
point_index = []
_point = False
k = []
for i in range(0, len(data_max), 50):
    ar = np.mean(data_max[i : i + 50])
    if ar > zero:
        k.append(i)
        _point = True
    if ar < zero and _point == True:
        point_index.append((k[0], i))
        k = []
        _point = False
    if _point == True and i >= len(data_max) - 20:
        point_index.append((k[0], len(data_max) - 1))
print(len(point_index))
print(point_index)

45
[(150, 400), (550, 900), (1000, 1350), (1450, 1800), (1950, 2250), (2400, 2750), (2850, 3200), (3300, 3650), (3800, 4100), (4250, 4600), (4700, 5050), (5150, 5500), (5650, 5950), (6100, 6450), (6550, 6900), (7000, 7350), (7500, 7850), (7950, 8300), (8400, 8750), (8900, 9200), (9350, 9700), (9800, 10150), (10250, 10600), (10750, 11100), (11200, 11550), (11650, 12000), (12150, 12450), (12600, 12950), (13050, 13400), (13500, 13850), (14000, 14350), (14450, 14800), (14900, 15250), (15400, 15700), (15850, 16150), (16300, 16650), (16750, 17100), (17250, 17550), (17700, 18050), (18150, 18500), (18600, 18950), (19100, 19400), (19550, 19900), (20000, 20350), (20500, 20800)]


In [14]:
for i in range(len(point_index)):
    print(point_index[i], "   ", point_index[i][1] - point_index[i][0])

(150, 400)     250
(550, 900)     350
(1000, 1350)     350
(1450, 1800)     350
(1950, 2250)     300
(2400, 2750)     350
(2850, 3200)     350
(3300, 3650)     350
(3800, 4100)     300
(4250, 4600)     350
(4700, 5050)     350
(5150, 5500)     350
(5650, 5950)     300
(6100, 6450)     350
(6550, 6900)     350
(7000, 7350)     350
(7500, 7850)     350
(7950, 8300)     350
(8400, 8750)     350
(8900, 9200)     300
(9350, 9700)     350
(9800, 10150)     350
(10250, 10600)     350
(10750, 11100)     350
(11200, 11550)     350
(11650, 12000)     350
(12150, 12450)     300
(12600, 12950)     350
(13050, 13400)     350
(13500, 13850)     350
(14000, 14350)     350
(14450, 14800)     350
(14900, 15250)     350
(15400, 15700)     300
(15850, 16150)     300
(16300, 16650)     350
(16750, 17100)     350
(17250, 17550)     300
(17700, 18050)     350
(18150, 18500)     350
(18600, 18950)     350
(19100, 19400)     300
(19550, 19900)     350
(20000, 20350)     350
(20500, 20800)     300


In [20]:
point_index = [
    [3560, 4780],
    [7590, 8950],
    [11600, 12800],
    [15350, 16700],
    [19500, 20550],
    [26720, 28000],
    [30460, 31500],
    [34100, 35380],
    [37830, 39100],
    [41500, 42800],
    [49150, 50300],
    [52720, 53850],
    [56390, 57950],
    [60850, 62450],
    [65375, 66700],
    [74000, 75800],
    [78550, 80100],
    [82750, 84250],
    [86950, 88400],
    [91160, 92850],
]

In [21]:
dist_folder = r"C:\Users\Nik\Desktop\скрининг\points"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"

folder_name = "p2"
d_name = "2_"


if os.path.exists(os.path.join(dist_folder, folder_name)):

    shutil.rmtree(dist_folder + "/" + folder_name)


os.makedirs(os.path.join(dist_folder, folder_name))


for i in range(len(point_index)):

    os.makedirs(os.path.join(dist_folder + "/" + folder_name, d_name + str(i + 1)))

In [25]:
main_folder = r"C:\Users\Nik\Desktop\скрининг\2-25С"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
file_list = np.array(natsorted(os.listdir(main_folder)))
print(dist_folder + str(file_list[1]))


for i in range(len(point_index)):

    dist = dist_folder + "/" + folder_name + "/" + d_name + str(i + 1)

    for j in range(point_index[i][0], point_index[i][1]):

        shutil.copy2(main_folder + str(file_list[j]), dist)

    print(d_name, i + 1, " ", point_index[i][1] - point_index[i][0])

C:/Users/Nik/Desktop/скрининг/points/Absorbance__1__13-57-24-779.rmn
2_ 1   1220
2_ 2   1360
2_ 3   1200
2_ 4   1350
2_ 5   1050
2_ 6   1280
2_ 7   1040
2_ 8   1280
2_ 9   1270
2_ 10   1300
2_ 11   1150
2_ 12   1130
2_ 13   1560
2_ 14   1600
2_ 15   1325
2_ 16   1800
2_ 17   1550
2_ 18   1500
2_ 19   1450
2_ 20   1690
